This will be a processing pipe for data from the SensorTile.box into proper input for the Madgwick algorithm


In [6]:
import pandas as pd
import numpy as np
import os

def extract_sensortile_data():
    index = 2
    dir = "./datasets/Exp{}/".format(index)
    files = os.listdir(dir)

    # extract data and skip first three rows
    acc_data = pd.read_csv(dir+files[0], skiprows=3) # add skiprows=3 for proper file formatting
    gyr_data = pd.read_csv(dir+files[1], skiprows=3)
    mag_data = pd.read_csv(dir+files[2], skiprows=3)

    # extract date information
    # date = acc_data.Date
    # df = pd.Series(date, dtype=np.datetime64)

    # compute sample frequency
    acc_data['Date']=pd.to_datetime(acc_data['Date'])
    v_sample_rate = acc_data.groupby(acc_data.Date.dt.second).count().Date.median()
    
    # add cumulative lapsed time column
    acc_data['TimeDiff']=(acc_data.Date-acc_data.Date[0])
    acc_data['TimeDiff']=acc_data['TimeDiff']/ np.timedelta64(1, 's')
    
    # extract only last three columns (X, Y, Z) of data 
    acc_data = acc_data[acc_data.columns[-3:]]
    gyr_data = gyr_data[gyr_data.columns[-3:]]
    mag_data = mag_data[mag_data.columns[-3:]]

    # convert into correct units
    acc_data = acc_data/1000 # [mg] --> [g]

    # remove effect of gravity to last column only
    # acc_data[acc_data.columns[1]] -= 1

    # gyr_data = gyr_data/1000
    mag_data = mag_data/1000 # [mGa] --> [G]

    data = pd.concat([gyr_data, acc_data, mag_data], axis=1)

    header = ["Gyroscope X (deg/s)", "Gyroscope Y (deg/s)", "Gyroscope Z (deg/s)", "Accelerometer X (g)", "Accelerometer Y (g)", "Accelerometer Z (g)", "Magnetometer X (G)", "Magnetometer Y (G)", "Magnetometer Z (G)"]

    data = data[1:]
    data.columns = header
    data.to_csv("experiment_dataset{}_CalInertialAndMag.csv".format(index))
    print("dataset expeiment_dataset{}_CalInertialAndMag.csv has been created".format(index))
    return v_sample_rate, data
v_sample_rate, data = extract_sensortile_data()
print("freq sample is {}".format(v_sample_rate))

dataset expeiment_dataset2_CalInertialAndMag.csv has been created
freq sample is 20.0


In [51]:
import pandas as pd
import numpy as np
import os

data = pd.read_csv("./datasets/Data.csv")
data = data.iloc[:-1]
data = data[data.columns[:-1]]


acc_data = data[data.columns[:3]]
gyr_data = data[data.columns[3:6]]
mag_data = data[data.columns[-3:]]
# gyr_data /= 1000 # converts mdps to dps
# acc_data /= 1000 # converts mg to g
# mag_data /= 1000 # converts mG to G

data = pd.concat([gyr_data, acc_data, mag_data], axis=1)

data /= 1000 # converts all units from mili to unit
header = ["Gyroscope X (deg/s)", "Gyroscope Y (deg/s)", "Gyroscope Z (deg/s)", "Accelerometer X (g)", "Accelerometer Y (g)", "Accelerometer Z (g)", "Magnetometer X (G)", "Magnetometer Y (G)", "Magnetometer Z (G)"]
data.columns = header
data['Accelerometer Z (g)'] += 1000
data.to_csv("example_Data_CalInertialAndMag.csv")
print("dataset created")
data

dataset created


,Gyroscope X (deg/s),Gyroscope Y (deg/s),Gyroscope Z (deg/s),Accelerometer X (g),Accelerometer Y (g),Accelerometer Z (g),Magnetometer X (G),Magnetometer Y (G),Magnetometer Z (G)
0,0.002,0.000,-0.009,-0.009,0.002,998.989,0.312,0.267,0.162
1,0.002,0.000,-0.009,-0.009,0.002,998.989,0.312,0.267,0.162
2,0.002,-0.001,-0.009,-0.008,0.003,998.990,0.319,0.258,0.135
3,0.002,-0.002,-0.009,-0.009,0.002,998.991,0.318,0.243,0.156
4,0.002,-0.002,-0.009,-0.010,0.002,998.991,0.334,0.262,0.151
...,...,...,...,...,...,...,...,...,...
796,0.002,-0.002,-0.008,-0.004,0.015,998.992,0.274,0.018,0.175
797,0.002,-0.002,-0.009,-0.005,0.015,998.992,0.264,0.006,0.172
798,0.002,-0.002,-0.009,-0.005,0.015,998.991,0.264,0.006,0.172
799,0.002,-0.001,-0.009,-0.003,0.015,998.990,0.271,0.027,0.192


,accX[mg],accY[mg],accZ[mg],gyroX[mdps],gyroY[mdps],gyroZ[mdps],magX[mG],magY[mG],magZ[mG],Unnamed: 9
0,-9,2,-1011,2.0,0.0,-9.0,312.0,267.0,162.0,NaN
1,-9,2,-1011,2.0,0.0,-9.0,312.0,267.0,162.0,NaN
2,-8,3,-1010,2.0,-1.0,-9.0,319.0,258.0,135.0,NaN
3,-9,2,-1009,2.0,-2.0,-9.0,318.0,243.0,156.0,NaN
4,-10,2,-1009,2.0,-2.0,-9.0,334.0,262.0,151.0,NaN
...,...,...,...,...,...,...,...,...,...,...
797,-5,15,-1008,2.0,-2.0,-9.0,264.0,6.0,172.0,NaN
798,-5,15,-1009,2.0,-2.0,-9.0,264.0,6.0,172.0,NaN
799,-3,15,-1010,2.0,-1.0,-9.0,271.0,27.0,192.0,NaN
800,-4,15,-1007,2.0,-1.0,-9.0,262.0,19.0,180.0,NaN


In [7]:
import pandas as pd
from datetime import datetime, timedelta
index = 5
dir = "./datasets/Exp{}/".format(index)
files = os.listdir(dir)

# extract data and skip first three rows
acc_data = pd.read_csv(dir+files[0], skiprows=3) # add skiprows=3 for proper file formatting
gyr_data = pd.read_csv(dir+files[1], skiprows=3)
mag_data = pd.read_csv(dir+files[2], skiprows=3)

date = acc_data.Date
df = pd.to_datetime(date)
# df.dt.total_seconds()
timedelta(df.dt.time.values[0]) - timedelta(datetime.now().time())


TypeError: unsupported type for timedelta days component: datetime.time

In [27]:
# set1 = date
set1['Date']=pd.to_datetime(set1['Date'])
v_sample_rate = set1.groupby(set1.Date.dt.second).count().mean()

set1['TimeDiff']=(set1.Date-set1.Date[0])
set1['TimeDiff']=set1['TimeDiff']/ np.timedelta64(1, 's')

KeyError: 'Date'

In [26]:
# print(date)
# date=pd.to_datetime(date)
v_sample_rate = date.groupby(date.dt.second).count().mean()
print("v_sample = {}".format(v_sample_rate))

date['TimeDiff']=(date-date[0])
date['TimeDiff']=date['TimeDiff']/ np.timedelta64(1, 's')
# print(date)

TypeError: cannot convert an object of type <class 'pandas.core.series.Series'> to a datetimelike index

In [24]:
# This is to test the data coming from Grace's sensor tile box
import pandas as pd
import os

os.listdir("./datasets")


def format_grace_st():
    # input the data file from Grace's ST
    # output: saved formatted data and sampling frequency of data
    data = pd.read_csv("./datasets/Data.csv")


    acc = data[['accX[mg]', 'accY[mg]', 'accZ[mg]']]
    gyr = data[['gyroX[mdps]', 'gyroY[mdps]', 'gyroZ[mdps]']]
    mag = data[['magX[mG]', 'magY[mG]','magZ[mG]']]

    # extract sampling rate, v_sample_rate
    date = data[data.columns[-1]]
    date = pd.to_datetime(date)
    v_sample_rate = date.groupby(date.dt.second).count().median()

    # reorganize data into proper format
    formatted_data = pd.concat([gyr, acc, mag], axis=1)

    # unit conversion
    formatted_data /= 1000

    # relabel header and save to file
    header = ["Gyroscope X (deg/s)", "Gyroscope Y (deg/s)", "Gyroscope Z (deg/s)", "Accelerometer X (g)", "Accelerometer Y (g)", "Accelerometer Z (g)", "Magnetometer X (G)", "Magnetometer Y (G)", "Magnetometer Z (G)"]
    formatted_data.columns = header
    formatted_data.to_csv("example_Grace_Data_CalInertialAndMag.csv")
    return v_sample_rate
